In [2]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re
import ast
import time
from typing import Optional
import requests_cache
from retry_requests import retry
import openmeteo_requests

In [5]:



        # --------------------------------------------------------------- #
        #                             WEBSCRAPING                         #
        # --------------------------------------------------------------- #

# URL de base
url = "https://www.af3v.org/resultats-des-recherches/?stv%5B1%5D=on&sact%5B9%5D=on&sact%5B6%5D=on&ssort=dista"
# Tri : voie verte lisse, vélo de route et roller

#@st.cache_resource
#def get_session():
    #return requests_cache.CachedSession('.http_cache', expire_after=86400)  # 24h

#@st.cache_data
def get_basic_fiches():
    #session = get_session()
   
    # Récupération des noms des fiches VV et les liens sur les différentes pages de résultats 

    data = {}
    index = 0

    for i in range(0,325,10):
        link = f"https://www.af3v.org/resultats-des-recherches/?stv%5B1%5D=on&sact%5B9%5D=on&sact%5B6%5D=on&ssort=dista&searchoffs={i}&blockOrLine=0"

        #html = session.get(link)
        html = requests.get(link)

        # récupérer le nom, les villes de début et fin et le lien de la fiche
        if html.status_code == 200:
            soup = BeautifulSoup(html.text, 'html.parser')
            cards = [div for div in soup.find_all("div", class_="openSansReg temCom198")] # Pour chaque élément div trouvé dans soup.find_all(...), il est gardé dans une liste.
            cards_town = [div for div in soup.find_all("div", class_="openSansReg s12w400 coc7c7c7")]
        

            for card, cardt in zip(cards, cards_town):      # si le nombre de résultats change surtout pour dernière page)
                var_1 = card.find('a')
                if var_1:
                    name = var_1.get_text(strip=True)
                    link_end = var_1['href']
                    link_full = "https://www.af3v.org" + link_end
                    trajet = cardt.get_text(strip=True)
                    words = re.search(r'De (.+?) à (.+)', trajet)
                    

                    if words:
                        start = words.group(1)
                        end = words.group(2)

                        data[index] = { 'Nom': name,
                                        'Lien': link_full,
                                        'Début': start,
                                        'Fin': end}
                        index += 1
    return data

#@st.cache_data
def get_fiches_characteristics(basic_data):
        # récupérer les autres infos caractéristiques en rentrant dans la fiche
    #session = get_session()

    data = basic_data.copy()

    for i in data:
        link_fiche = data[i]['Lien']
        #html_fiche = session.get(link_fiche)
        html_fiche = requests.get(link_fiche)

        if html_fiche.status_code == 200:
            soup_fiche = BeautifulSoup(html_fiche.text, 'html.parser')
            caracteristics_dist = [div for div in soup_fiche.find_all("div", class_="co767676 openSansMed s14w400")]
            if len(caracteristics_dist) >= 4:
                distance = caracteristics_dist[0].get_text(strip=True)
                type_voie = caracteristics_dist[1].get_text(strip=True)
                nature = caracteristics_dist[2].get_text(strip=True)
                revetement = caracteristics_dist[3].get_text(strip=True)
                data[i].update({ 'Distance': distance,
                            'Type voie': type_voie,
                            'Nature voie': nature,
                            'Revêtement': revetement})
    return data

#@st.cache_data
def get_fiches_coordinates(data_with_chars):    

    #print("🗺️  Récupération des coordonnées...")

    #session = get_session()

    data = data_with_chars.copy()

    for i in data:
        link_fiche = data[i]['Lien']
        #html_fiche = session.get(link_fiche)
        html_fiche = requests.get(link_fiche)

        if html_fiche.status_code == 200:
            soup_fiche = BeautifulSoup(html_fiche.text, 'html.parser')
            car_map = [div for div in soup_fiche.find_all("iframe", id="carte_voies")]
            for map in car_map:
                carte = map["data-url"]
                coords_all = re.search(r'(?<=bbox=)[^&]*', carte) # cherche tout ce qui est après = et avant &
                if coords_all:
                    coords_str = coords_all.group() # récupérer la str extraite
                    coords = coords_str.split(',') # séparer par ,
                    coord_start = (coords[0], coords[1])
                    coord_end = (coords[2], coords[3])
                    data[i].update({ 'Carte': carte,
                                    'Coordonnées de début': coord_start,
                                    'Coordonnées de fin': coord_end})                        

    return data

#@st.cache_data                   
def scraping():
    
    # Étape 1
    #progress_bar = st.progress(0)
    #status_text = st.empty()
    
    #status_text.text("Récupération des fiches de base...")
    basic_data = get_basic_fiches()
    #progress_bar.progress(33)
    
    # Étape 2
    #status_text.text("Ajout des caractéristiques...")
    data_with_chars = get_fiches_characteristics(basic_data)
    #progress_bar.progress(66)
    
    # Étape 3
    #status_text.text("Ajout des coordonnées...")
    final_data = get_fiches_coordinates(data_with_chars)
    #progress_bar.progress(100)
    
    #status_text.text(f"L'attente est finie : {len(final_data)} fiches viennent d'être récupérées sur le site de l'Association Française pour le développement des Véloroutes et Voies Vertes.")
    return final_data

data = scraping()

#@st.cache_data()
def convert_en_df(data):
    df = pd.DataFrame.from_dict(data, orient='index')

    def midpoint(lat1, lon1, lat2, lon2):
        mid_lat = (lat1 + lat2) / 2
        mid_lon = (lon1 + lon2) / 2
        return mid_lat, mid_lon

    def calculate_midpoint_voie_verte(row):
        try:
            coord_debut = row['Coordonnées de début']
            coord_fin = row['Coordonnées de fin']
            
            # Convertir selon le type
            if isinstance(coord_debut, str):
                coord_debut = coord_debut.strip('()')
                lat_debut, lon_debut = map(float, coord_debut.split(','))
            else:
                lat_debut, lon_debut = float(coord_debut[0]), float(coord_debut[1])
            
            if isinstance(coord_fin, str):
                coord_fin = coord_fin.strip('()')
                lat_fin, lon_fin = map(float, coord_fin.split(','))
            else:
                lat_fin, lon_fin = float(coord_fin[0]), float(coord_fin[1])
            
            return midpoint(lat_debut, lon_debut, lat_fin, lon_fin)
        except Exception as e:
            print(f"Erreur: {e}")
            return None
    
    df['Coordonnées milieu'] = df.apply(calculate_midpoint_voie_verte, axis=1)

    def safe_literal_eval(val):
        if isinstance(val, str):
            try:
                return ast.literal_eval(val)
            except Exception as e:
                print(f"Erreur pour la valeur {val}: {e}")
                return None
        else:
            return val  # déjà un tuple ou autre type valide
    
    df["Coordonnées milieu"] = df["Coordonnées milieu"].apply(safe_literal_eval)
    df["Coordonnées de début"] = df["Coordonnées de début"].apply(safe_literal_eval)
    df["Coordonnées de fin"] = df["Coordonnées de fin"].apply(safe_literal_eval)
    df['Distance'] = df['Distance'].str.replace('km', '', regex=False).str.replace(',', '.').astype(float)
    df['Début'] = df['Début'].str.capitalize()
    df['Fin'] = df['Fin'].str.capitalize()
    df['% Couverture correspondant (200m)'] = round((200 / (df['Distance'] * 1000) * 100),2)

    return df

df = convert_en_df(data)

#@st.cache_data()
def categorize_nature(voie) -> str :
    voie = str(voie).lower()
    if 'rivière' in voie or 'canal' in voie or 'chemin de halage' in voie:
        return 'Bord de rivière ou canal'
    elif 'sentier' in voie or 'chemin' in voie:
        return 'Bord de sentier'
    elif 'route' in voie:
        return 'Bord de route'
    elif 'piste' in voie:
        return 'Piste cyclable'
    elif 'chemin de halage' in voie:
        return 'Chemin de halage'
    elif 'ancienne' in voie:
        return 'Ancienne ligne ferrovière'
    elif 'verte' in voie:
        return 'Voie verte'
    elif 'ferrée'in voie:
        return 'Bord de voie ferrée'
    else:
        return 'Voie verte'
    
df['Groupes nature voie'] = df['Nature voie'].apply(categorize_nature)




In [6]:


        # --------------------------------------------------------------- #
        #                    COMPLETION DONNEES SOLAIRES                  #
        # --------------------------------------------------------------- #

#@st.cache_data
def solar_info_annuel(coord_tuple):
    coord_string = str(coord_tuple)
    coord_string = coord_string.strip('()')  # Retire les parenthèses si présentes
    lon, lat = map(float, coord_string.split(','))

    # Délai de 1-2 secondes entre chaque requête
    time.sleep(1)
    
    # Setup the Open-Meteo API client with cache and retry on error
    cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
    retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
    openmeteo = openmeteo_requests.Client(session = retry_session)

    # Paramètres à renseigner
    url = "https://archive-api.open-meteo.com/v1/archive"
    params = {
        "latitude": lat,
        "longitude": lon,
        "start_date": f"2024-01-01",
        "end_date": f"2024-12-31",
        "daily": ["sunshine_duration", "temperature_2m_mean"],
        "hourly": "shortwave_radiation",
        "timezone": "Europe/Berlin"
    }
    responses = openmeteo.weather_api(url, params=params)

    response = responses[0]
    

    # Process hourly data
    hourly = response.Hourly()
    hourly_shortwave_radiation = hourly.Variables(0).ValuesAsNumpy()

    hourly_data = {"date": pd.date_range(
        start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
        end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
        freq = pd.Timedelta(seconds = hourly.Interval()),
        inclusive = "left"
    )}

    hourly_data["shortwave_radiation"] = hourly_shortwave_radiation

    hourly_dataframe = pd.DataFrame(data = hourly_data)


    # Nettoyage pour exploitation des données du df Hourly
    english_months = ['January', 'February', 'March', 'April', 'May', 'June',
                        'July', 'August', 'September', 'October', 'November', 'December']

    hourly_dataframe['month'] = hourly_dataframe['date'].dt.month.apply(lambda x: english_months[x-1])
    hourly_dataframe['month_num'] = hourly_dataframe['date'].dt.month

    hourly_dataframe = hourly_dataframe.iloc[2:]

    df_ghi = hourly_dataframe.groupby(['month', 'month_num']).agg(
        ghi_mean_W_m2=('shortwave_radiation', 'mean')
    )

    df_ghi = df_ghi.sort_values(by='month_num')


    # Process daily data
    daily = response.Daily()
    daily_sunshine_duration = daily.Variables(0).ValuesAsNumpy()
    daily_temperature_2m_mean = daily.Variables(1).ValuesAsNumpy()

    daily_data = {"date": pd.date_range(
        start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
        end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
        freq = pd.Timedelta(seconds = daily.Interval()),
        inclusive = "left"
    )}

    daily_data["sunshine_duration"] = daily_sunshine_duration
    daily_data["temperature_2m_mean"] = daily_temperature_2m_mean

    daily_dataframe = pd.DataFrame(data = daily_data)


    # Nettoyage pour exploitation des données du df daily
    daily_dataframe['sunshine_duration_hours'] = daily_dataframe['sunshine_duration'] / 3600
    daily_dataframe.drop(columns='sunshine_duration', inplace=True)

    daily_dataframe = daily_dataframe.iloc[1:]

    daily_dataframe['month'] = daily_dataframe['date'].dt.month.apply(lambda x: english_months[x-1])
    daily_dataframe['month_num'] = daily_dataframe['date'].dt.month
    daily_dataframe['days_in_month'] = daily_dataframe['date'].dt.daysinmonth



    df_weather = daily_dataframe.groupby(['month', 'month_num']).agg(
        temp_mean_c=('temperature_2m_mean', 'mean'),
        sunshine_duration_mean_hour=('sunshine_duration_hours', 'mean'),
        days_in_month=('days_in_month', 'first')  
    )


    df_weather = df_weather.sort_values(by='month_num')

    # Concaténation des dfs

    df = pd.merge(df_weather, df_ghi, on='month')
    

    # ==================#
    #     Panneau       #
    #  Photovoltaïques  #
    # ==================#

    # --- Paramètres spécifiques à l'installation ---
    # Ajout de paramètres types, servant d'exemples, 
    EFFICIENCY_PANEL = 0.20  # Exemple : rendement de 20 % (par ex. pour un panneau de 400W, diviser par sa surface en m²)

    ANGLE_ORIENTATION_FACTOR = 1.00 # paramètre crucial pour estimer le rendement énergétique réel d’un système photovoltaïque, car il prend en compte l’impact de l’alignement physique de l’installation par rapport au soleil.

    TEMP_COEFFICIENT_PMAX = 0.004  # Typique : 0.004 soit 0,4 % de perte de puissance par °C 

    SYSTEM_LOSS_FACTOR = 0.85      # Tient compte des pertes du système : onduleur, câblage, saleté, ombrage, etc. (typiquement entre 0.75 et 0.90)

    DELTA_TEMP_PANEL = 20          # Écart estimé entre la température du panneau et la température ambiante (en °C)

    # =================#
    #     Formules     #
    # =================#

    # 1. Calcul de l’irradiance journalière en kWh/m²/jour
    # Ce calcul suppose que ghi_mean_W_m2 et sunshine_duration_mean_hour représentent des moyennes journalières pour le mois.
    df['ghi_journalier_kWh_m2_jour'] = (df['ghi_mean_W_m2'] * df['sunshine_duration_mean_hour']) / 1000

    # 2. Estimation de la température de fonctionnement des panneaux
    # La température du panneau est généralement supérieure à la température ambiante
    df['panel_temp_c'] = df['temp_mean_c'] + DELTA_TEMP_PANEL

    # 3. Calcul du facteur de correction de température
    # Les performances diminuent lorsque la température dépasse 25°C
    df['correction_temp'] = 1 - (df['panel_temp_c'] - 25) * TEMP_COEFFICIENT_PMAX

    # On s’assure que correction_temp reste dans des bornes raisonnables (évite des gains irréalistes ou pertes trop fortes)
    df['correction_temp'] = df['correction_temp'].clip(lower=0.95, upper=1.05)

    # 4. Calcul de la production d’énergie journalière par m² (kWh/jour/m²)
    # On combine tous les facteurs :
    # GHI_journalier * Rendement * Facteur d’orientation * Correction température * Pertes système
    df['energie_jour_kWh/j/m2'] = df['ghi_journalier_kWh_m2_jour'] * \
                                EFFICIENCY_PANEL * \
                                ANGLE_ORIENTATION_FACTOR * \
                                df['correction_temp'] * \
                                SYSTEM_LOSS_FACTOR

    # Arrondi à 2 décimales pour l’affichage
    df['energie_jour_kWh/j/m2'] = round(df['energie_jour_kWh/j/m2'], 2)

    # 5. Calcul de la production mensuelle d’énergie par m² (kWh/mois/m²)
    # On multiplie la production journalière par le nombre de jours dans le mois
    df['energie_mois_kWh/mois/m2'] = round(df['energie_jour_kWh/j/m2'] * df['days_in_month'], 2)

    return round(df['energie_mois_kWh/mois/m2'].sum(), 2)
    
df['Energie produite annuelle (kWh)'] = df['Coordonnées milieu'].apply(solar_info_annuel)
df['Énergie produite annuelle (kWh / 860m2 de panneau)'] = round(df['Energie produite annuelle (kWh)'].apply(lambda x: x*860))



In [19]:
        # --------------------------------------------------------------- #
        #                         AJOUT SOLAR SCORE                       #
        # --------------------------------------------------------------- #


#@st.cache_data()
#def load_data():
    #df_solarscore = pd.read_csv("datasets/df_solarscore.csv")
    #return df_solarscore

#df_solarscore = load_data()

df_solarscore = pd.read_csv("C:\\Users\\mimia\\Desktop\\projet_3\\datasets\\df_solarscore.csv")

datafinal = pd.merge(df, df_solarscore, how = 'right', on='Nom')

In [18]:
datafinal.head

<bound method NDFrame.head of                                                    Nom  \
0                  Voie Verte du Chemin d’Encochebérot   
1                 Voie Verte du canal de Saint-Martory   
2            Voie Verte de la Masse Pont-du-Casse-Agen   
3    Voie Verte des trois ponts Castelnau d’Estréte...   
4            Voie Verte des Berges de l'Ill à Mulhouse   
..                                                 ...   
318  V61 - Véloroute du Léman au Mont-Blanc : de Ge...   
319  Le Canal des deux mers à vélo (V80), en Tarn-e...   
320  Véloroute Tour du Bassin d’Arcachon (EV1 Vélod...   
321                                   Véloroute du Lin   
322                                    Piste des forts   

                                                  Lien  \
0    https://www.af3v.org/les-voies-vertes/voies/12...   
1    https://www.af3v.org/les-voies-vertes/voies/12...   
2    https://www.af3v.org/les-voies-vertes/voies/11...   
3    https://www.af3v.org/les-voies-verte

In [20]:
datafinal.to_csv("df_voies_vertes_MAJ_AOUT.csv")